I. Theoretical session:
1. Could you list out some limitation of MapReduce?
2. Provide a high level comparison of Apache Hadoop and Apache Spark.
3. What are the advantages of Apache Spark?
4. Provide a comparison of RDD and DataFrame in Spark.  

II. You are given a file `appl_stock.csv`, please carry out the following tasks:

1. Read this file by PySpark. Print out the schema.
2. Create new columns of combining the High, Low, Close and Adj Close as follow `[High, Low, Close, Adj Close]`.
3. Create a new column which computes the average price of High and Low prices.
4. Create a new column which computes the amount of money based on the formula `Volume * Adj Close`.
3. Using `groupby` and `year()` function to compute the average closing price per year.


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ops').getOrCreate()

In [3]:
from google.colab import files
files.upload()

Saving appl_stock.csv to appl_stock.csv


{'appl_stock.csv': b'Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004\n2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265\n2010-01-08,210.299994,212.000006,209.06000500000002,211.98000499999998,111902700,27.464034\n2010-01-11,212.79999700000002,213.000002,208.450005,210.11000299999998,115557400,27.221758\n2010-01-12,209.18999499999998,209.76999500000002,206.419998,207.720001,148614900,26.91211\n2010-01-13,207.870005,210.92999500000002,204.099998,210.650002,151473000,27.29172\n2010-01-14,210.11000299999998,210.45999700000002,209.020004,209.43,108223500,27.133657\n2010-01-15,210.92999500000002,211.59999700000003,205.869999,205.93,148516900,26.680197999999997\n2010-01-19,208.330002,215.18999900000003,207.240004,215.039995,182501900,27.

In [5]:
##1 Print schema
df = spark.read.csv('/content/appl_stock.csv', inferSchema=True, header=True)
df.show(truncate=False)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|Date      |Open              |High              |Low               |Close             |Volume   |Adj Close         |
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998        |214.499996        |212.38000099999996|214.009998        |123432400|27.727039         |
|2010-01-05|214.599998        |215.589994        |213.249994        |214.379993        |150476200|27.774976000000002|
|2010-01-06|214.379993        |215.23            |210.750004        |210.969995        |138040000|27.333178000000004|
|2010-01-07|211.75            |212.000006        |209.050005        |210.58            |119282800|27.28265          |
|2010-01-08|210.299994        |212.000006        |209.06000500000002|211.98000499999998|111902700|27.464034         |
|2010-01-11|212.79999700000002|213.000002        |208.45

In [7]:
##2 Create new columns of combining the High, Low, Close and Adj Close
from pyspark.sql.functions import col, year
df = df.withColumn("Combined_Column", col("High") + col("Low") + col("Close") + col("Adj Close"))
df.show()


+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|  Combined_Column|
+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|       668.617034|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|       670.994957|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|       664.283177|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|658.9126610000001|
|2010-01-08|        210.299994|        212.00000

In [9]:
##3 Create a new column for average price of High and Low prices
df = df.withColumn("Average_Price", (col("High") + col("Low")) / 2)
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|  Combined_Column|     Average_Price|
+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|       668.617034|       213.4399985|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|       670.994957|214.41999399999997|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|       664.283177|        212.990002|
|2010-01-07|            211.75|        212.000006|        209.050005| 

In [10]:
##4 Create a new column which computes the amount of money based on the formula Volume * Adj Close
df = df.withColumn("Amount_Money", col("Volume") * col("Adj Close"))
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+------------------+--------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|  Combined_Column|     Average_Price|        Amount_Money|
+----------+------------------+------------------+------------------+------------------+---------+------------------+-----------------+------------------+--------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|       668.617034|       213.4399985|   3.4224149686636E9|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|       670.994957|214.41999399999997|4.1794728435712004E9|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|   

In [11]:
##5 Using groupby and year() function to compute the average closing price per year.
df1 = df.groupBy(year("Date").alias("Year")).agg({"Close": "avg"})
df1.show()


+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



III. You are given a data `customer_churn.csv`, which describes the churn status in clients of a marletting agency. As a data scientist, you are required to create a machine learning model **in Spark** that will help predict which customers will churn (stop buying their service). A short description of the data is as follow:
```
Name : Name of the latest contact at Company
Age: Customer Age
Total_Purchase: Total Ads Purchased
Account_Manager: Binary 0=No manager, 1= Account manager assigned
Years: Totaly Years as a customer
Num_sites: Number of websites that use the service.
Onboard_date: Date that the name of the latest contact was onboarded
Location: Client HQ Address
Company: Name of Client Company
```

1. Read, print the schema and check out the data to set the first sight of the data.
2. Format the data according to `VectorAssembler`, which is supported in MLlib of PySpark.
3. Split the data into train/test data, and then fit train data to the logistic regression model.
4. Evaluate the results and compute the AUC.

In [12]:
from google.colab import files
files.upload()

Saving customer_churn.csv to customer_churn.csv


{'customer_churn.csv': b'Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn\nCameron Williams,42.0,11066.8,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1\nKevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 17756",Wilson PLC,1\nEric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1\nPhillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1\nCynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1\nJessica Williams,48.0,10356.02,0,5.12,8.0,2009-03-03 23:13:37,"6187 Olson Mountains East Vincentborough, PR 74359",Kelly-Warren,1\nEric Butler,44.0,11331.58,1,5.23,11.0,2016-12-05 03:35:43,"4846 Savannah Road West Justin, IA 87713-3460",Reynolds-Sheppard,1\nZachary Wals

In [19]:
##1 Read, print the schema and check out the data to set the first sight of the data.
df = spark.read.csv('/content/customer_churn.csv', inferSchema=True, header=True)
df.show(truncate=False)
print('Description:')
df.printSchema()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------------------------------------------+----------------------------+-----+
|Names              |Age |Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date       |Location                                                |Company                     |Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------------------------------------------+----------------------------+-----+
|Cameron Williams   |42.0|11066.8       |0              |7.22 |8.0      |2013-08-30 07:00:40|10265 Elizabeth Mission Barkerburgh, AK 89518           |Harvey LLC                  |1    |
|Kevin Mueller      |41.0|11916.22      |0              |6.5  |11.0     |2013-08-13 00:38:46|6157 Frank Gardens Suite 019 Carloshaven, RI 17756      |Wilson PLC                  |1    |
|Eric Lozano        |38.0|12884.75      |0              |6.67 |12.0   

In [22]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [27]:
##2 format the data according to VectorAssembler, which is supported in MLlib of PySpark.
feature_columns = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
df_assembled = assembler.transform(df).select('features', 'Company', 'Churn')  # Assuming 'Churn' is the target column
df_assembled.show()


+--------------------+--------------------+-----+
|            features|             Company|Churn|
+--------------------+--------------------+-----+
|[42.0,11066.8,0.0...|          Harvey LLC|    1|
|[41.0,11916.22,0....|          Wilson PLC|    1|
|[38.0,12884.75,0....|Miller, Johnson a...|    1|
|[42.0,8010.76,0.0...|           Smith Inc|    1|
|[37.0,9191.58,0.0...|          Love-Jones|    1|
|[48.0,10356.02,0....|        Kelly-Warren|    1|
|[44.0,11331.58,1....|   Reynolds-Sheppard|    1|
|[32.0,9885.12,1.0...|          Singh-Cole|    1|
|[43.0,14062.6,1.0...|           Lopez PLC|    1|
|[40.0,8066.94,1.0...|       Reed-Martinez|    1|
|[30.0,11575.37,1....|Briggs, Lamb and ...|    1|
|[45.0,8771.02,1.0...|    Figueroa-Maynard|    1|
|[45.0,8988.67,1.0...|     Abbott-Thompson|    1|
|[40.0,8283.32,1.0...|Smith, Kim and Ma...|    1|
|[41.0,6569.87,1.0...|Snyder, Lee and M...|    1|
|[38.0,10494.82,1....|      Sanders-Pierce|    1|
|[45.0,8213.41,1.0...|Andrews, Adams an...|    1|


In [43]:
##3 Split the data into train/test data, and then fit train data to the logistic regression model.
train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=42)
log_reg = LogisticRegression(featuresCol='features', labelCol='Churn')
model = log_reg.fit(train_data)

coefficients = model.coefficients
intercept = model.intercept

print("Coefficients: ", coefficients)
print("Intercept: ", intercept)

Coefficients:  [0.06058842660840501,5.1725150114147306e-05,0.46349688676333073,0.5432876344567339,1.1488837959327736]
Intercept:  -18.826551620434515


In [47]:
##4 Evaluate the results and compute the AUC.
predict = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(labelCol='Churn', metricName='areaUnderROC')
auc = evaluator.evaluate(predict)

print("AUC:", auc)

AUC: 0.9367816091954025
